### Random Forest Classifier

In [169]:
import pandas as pd
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test = pd.read_csv("../data/processed/X_test.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")

In [170]:
from sklearn.ensemble import RandomForestClassifier

X_id = X_test["PassengerId"]
y_id = y_test["PassengerId"]
X_train = X_train.drop(columns=["PassengerId"])
X_test = X_test.drop(columns=["PassengerId"])
y_train = y_train.drop(columns=["PassengerId"])
y_test = y_test.drop(columns=["PassengerId"])
import numpy as np
y_test = np.array(y_test["Survived"])
y_train = np.array(y_train["Survived"])

rf = RandomForestClassifier(n_estimators=100, random_state=42)
X_train

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,-1.637881,-1.935824,-0.474161,1.998853,0.980998,1,0,1
1,0.803267,-0.035688,-0.474161,-0.479327,-0.469634,1,0,1
2,0.803267,-2.166255,0.348687,0.759763,-0.406136,0,0,1
3,-0.417307,0.522114,0.348687,1.998853,-0.080232,1,0,1
4,-0.417307,1.059788,0.348687,0.759763,-0.109651,1,0,1
...,...,...,...,...,...,...,...,...
618,0.803267,-0.630044,-0.474161,-0.479327,-0.474455,0,0,1
619,-1.637881,-0.001253,-0.474161,-0.479327,-0.016489,1,0,1
620,0.803267,0.906167,1.171535,-0.479327,-0.347787,1,0,1
621,-1.637881,-1.167718,0.348687,1.998853,1.729074,0,0,1


In [171]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [172]:
rf.score(X_test, y_test)

0.7798507462686567

In [173]:
from sklearn.metrics import precision_score
precision_score(y_test, rf.predict(X_test))

0.7954545454545454

In [174]:
a = rf.predict(X_test)

In [175]:
b = pd.DataFrame(a, columns=["Survived"])
b.mean()

Survived    0.328358
dtype: float64

In [176]:
final = pd.DataFrame(y_id)
final["Survived"] = pd.DataFrame(a)

In [177]:
final

,PassengerId,Survived
0,710,0
1,440,0
2,841,0
3,721,1
4,40,1
...,...,...
263,822,0
264,634,0
265,457,0
266,501,0


In [178]:
validate = pd.read_csv("../data/raw/validate.csv")

In [179]:
def clean_titanic(df):
    df = df.drop(
        columns=[
            "Name",
            "Ticket",
            "Cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)

    return df

In [180]:
validate = clean_titanic(validate)

In [181]:
import pandas as pd
from sklearn import model_selection, impute, preprocessing
from sklearn.experimental import enable_iterative_imputer
def process(df):
    
    num_cols = [
        "Age",
        "Fare",
    ]

    imputer = impute.IterativeImputer()
    df.loc[:,num_cols] = imputer.fit_transform(df[num_cols])


    cols = "Pclass,Age,SibSp,Parch,Fare".split(",")
    sca = preprocessing.StandardScaler()

    df.loc[:,cols] = sca.fit_transform(df[cols])

    return df

In [182]:
validate = process(validate)
validate

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,892,0.873482,0.366672,-0.499470,-0.400248,-0.499979,1,1,0
1,893,0.873482,1.354422,0.616992,-0.400248,-0.514834,0,0,1
2,894,-0.315819,2.539722,-0.499470,-0.400248,-0.466687,1,1,0
3,895,0.873482,-0.225979,-0.499470,-0.400248,-0.485050,1,0,1
4,896,0.873482,-0.621079,0.616992,0.619896,-0.420109,0,0,1
...,...,...,...,...,...,...,...,...,...
413,1305,0.873482,-0.168278,-0.499470,-0.400248,-0.496023,1,0,1
414,1306,-1.505120,0.722262,-0.499470,-0.400248,1.310701,0,0,0
415,1307,0.873482,0.682752,-0.499470,-0.400248,-0.510355,1,0,1
416,1308,0.873482,-0.168278,-0.499470,-0.400248,-0.496023,1,0,1


In [183]:
final_1 = pd.DataFrame(validate["PassengerId"])
validate = validate.drop(columns=["PassengerId"])


In [184]:
final_2 = rf.predict(validate)

In [185]:
final_1["Survived"] = pd.DataFrame(final_2)

In [186]:
from src import src_path
from os import path

In [187]:
src_path

'/home/gbpagano/ihm/titanic/notebooks'

In [188]:
final_1.to_csv(path.join(src_path,"random_forest.csv"), header=True)

In [189]:
sub = pd.Series(final_2, index=final_1["PassengerId"], name="Survived")

In [190]:
sub.to_csv(path.join(src_path,"random_forest2.csv"), header=True)
